In [0]:
# VGGNet on MNIST dataset...
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
# fix dimension ordering issue
from keras import backend as K


In [0]:

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][channels] -- I am using tensorflow.
X_train = X_train.reshape(X_train.shape[0], 28,28,1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28,28,1).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255.0
X_test = X_test / 255.0
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
inputShape = (28,28,1)


# define a LeNet CNN model in keras
def miniVGG_model():
    # create model
    model = Sequential()
    # first set of CONV => RELU => CONV => RELU=> POOL layers
    model.add(Conv2D(32, (3,3), padding="same",input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
  
    # second set of CONV => RELU => POOL layers
    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
   
    # first (and only) set of FC => RELU layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.50))
    # softmax classifier
    model.add(Dense(num_classes))
    model.add(Activation("softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


In [10]:
# build the model
model = miniVGG_model()
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

Train on 60000 samples, validate on 10000 samples
Epoch 1/3
60000/60000 [==============================] - 387s 6ms/step - loss: 0.1770 - acc: 0.9478 - val_loss: 0.0462 - val_acc: 0.9847
Epoch 2/3
39800/60000 [==================>...........] - ETA: 2:02 - loss: 0.0668 - acc: 0.9792

60000/60000 [==============================] - 386s 6ms/step - loss: 0.0636 - acc: 0.9806 - val_loss: 0.0349 - val_acc: 0.9879
Epoch 3/3
60000/60000 [==============================] - 386s 6ms/step - loss: 0.0459 - acc: 0.9857 - val_loss: 0.0401 - val_acc: 0.9870
CNN Error: 1.30%
